# #6 Streak Leaderboard

In [1]:
import pandas as pd

In [2]:
lesson_completion = (
    pd.read_parquet("LessonStreaks.parquet")
    .drop_duplicates(["user_id", "date"]) # drop instance of users completing multiple lessons on the same day
    .sort_values(["user_id", "date"])   # sort sequence of events by user ID and date
)
lesson_completion["date"] = pd.to_datetime(lesson_completion['date'])
lesson_completion.head()

,id,lesson_id,date,user_id,user_name
9330,7839783,63658156,2025-05-01,8226413,Aeris Stone
41178,7872349,63658157,2025-05-05,8226413,Aeris Stone
45072,7876346,63658171,2025-05-06,8226413,Aeris Stone
51344,7882741,63658176,2025-05-07,8226413,Aeris Stone
123456,7956420,64983797,2025-05-21,8226413,Aeris Stone


In [3]:
lesson_completion["streak"] = lesson_completion.groupby("user_id")["date"].diff().ne(pd.Timedelta(days=1)).cumsum()

streaks = lesson_completion.groupby(["user_id", "user_name", "streak"]).agg(
    streak_len = ("date", "count"),
    start = ("date", "min"),
    end = ("date", "max")
).reset_index()

In [4]:
active_date = pd.to_datetime("2025-09-28")
streaks.query("end >= @active_date").sort_values(["streak_len"], ascending=False)

,user_id,user_name,streak,streak_len,start,end
2419,84071240,Stan Pancake,2420,151,2025-05-01,2025-09-28
32,16814590,Tom Parmesan,33,121,2025-05-31,2025-09-28
14987,215518363,Chuck Butternut,14988,120,2025-06-01,2025-09-28
14074,213142099,Helen Melon,14075,96,2025-06-25,2025-09-28
14739,215121049,Gertrude Strudel,14740,88,2025-07-03,2025-09-28
...,...,...,...,...,...,...
39158,232458767,Vincent Dixon,39159,1,2025-09-28,2025-09-28
39175,232472960,Luna Malfoy,39176,1,2025-09-28,2025-09-28
39176,232473696,Quinn Rivera,39177,1,2025-09-28,2025-09-28
39177,232473825,Tchalla Zidane,39178,1,2025-09-28,2025-09-28
